In [ ]:
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout,Input, AveragePooling2D, Activation,Conv2D, MaxPooling2D, BatchNormalization,Concatenate
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


In [ ]:

tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
#data
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test)=cifar10.load_data()
y_train_cat=to_categorical(y_train,10)
y_test_cat=to_categorical(y_test,10)

170498071/170498071 [==============================] - 13s 0us/step


In [ ]:

resnet50 = K.applications.ResNet50(weights='imagenet',include_top=False,input_tensor=K.Input(shape=(224, 224, 3)))

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
for layer in resnet50.layers[:]:
   layer.trainable = False

In [ ]:
for i, layer in enumerate(resnet50.layers):
  print(i,layer.name,'-',layer.trainable)

0 input_1 - False
1 conv1_pad - False
2 conv1_conv - False
3 conv1_bn - False
4 conv1_relu - False
5 pool1_pad - False
6 pool1_pool - False
7 conv2_block1_1_conv - False
8 conv2_block1_1_bn - False
9 conv2_block1_1_relu - False
10 conv2_block1_2_conv - False
11 conv2_block1_2_bn - False
12 conv2_block1_2_relu - False
13 conv2_block1_0_conv - False
14 conv2_block1_3_conv - False
15 conv2_block1_0_bn - False
16 conv2_block1_3_bn - False
17 conv2_block1_add - False
18 conv2_block1_out - False
19 conv2_block2_1_conv - False
20 conv2_block2_1_bn - False
21 conv2_block2_1_relu - False
22 conv2_block2_2_conv - False
23 conv2_block2_2_bn - False
24 conv2_block2_2_relu - False
25 conv2_block2_3_conv - False
26 conv2_block2_3_bn - False
27 conv2_block2_add - False
28 conv2_block2_out - False
29 conv2_block3_1_conv - False
30 conv2_block3_1_bn - False
31 conv2_block3_1_relu - False
32 conv2_block3_2_conv - False
33 conv2_block3_2_bn - False
34 conv2_block3_2_relu - False
35 conv2_block3_3_conv - 

In [ ]:
#resnet50,teacher
model = K.models.Sequential()
model.add(K.layers.Lambda(lambda x: tf.image.resize(x,(224, 224))))
model.add(resnet50)
model.add(Flatten())
model.add(K.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(lr=2e-5),metrics=['accuracy'])
history1=model.fit(x_train,y_train_cat, batch_size=128,epochs=3,validation_data=(x_test,y_test_cat))
evaluation = model.evaluate(x_test, y_test_cat)
print('Test Accuracy: {}'.format(evaluation[1]))

model.save("resnet50-cifar10.h5")
model.summary()

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/3
391/391 [==============================] - 209s 528ms/step - loss: 0.6455 - accuracy: 0.7909 - val_loss: 0.5606 - val_accuracy: 0.8198
Epoch 2/3
391/391 [==============================] - 205s 524ms/step - loss: 0.3537 - accuracy: 0.8824 - val_loss: 0.4957 - val_accuracy: 0.8383
Epoch 3/3
313/313 [==============================] - 35s 108ms/step - loss: 0.4711 - accuracy: 0.8500
Test Accuracy: 0.8500000238418579
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)            

In [ ]:
##finedtuned
#https://towardsdatascience.com/deep-learning-using-transfer-learning-python-code-for-resnet50-8acdfb3a2d38
for layer in resnet50.layers[:]:
   layer.trainable = True

In [ ]:
for i, layer in enumerate(resnet50.layers):
  print(i,layer.name,'-',layer.trainable)

0 input_1 - True
1 conv1_pad - True
2 conv1_conv - True
3 conv1_bn - True
4 conv1_relu - True
5 pool1_pad - True
6 pool1_pool - True
7 conv2_block1_1_conv - True
8 conv2_block1_1_bn - True
9 conv2_block1_1_relu - True
10 conv2_block1_2_conv - True
11 conv2_block1_2_bn - True
12 conv2_block1_2_relu - True
13 conv2_block1_0_conv - True
14 conv2_block1_3_conv - True
15 conv2_block1_0_bn - True
16 conv2_block1_3_bn - True
17 conv2_block1_add - True
18 conv2_block1_out - True
19 conv2_block2_1_conv - True
20 conv2_block2_1_bn - True
21 conv2_block2_1_relu - True
22 conv2_block2_2_conv - True
23 conv2_block2_2_bn - True
24 conv2_block2_2_relu - True
25 conv2_block2_3_conv - True
26 conv2_block2_3_bn - True
27 conv2_block2_add - True
28 conv2_block2_out - True
29 conv2_block3_1_conv - True
30 conv2_block3_1_bn - True
31 conv2_block3_1_relu - True
32 conv2_block3_2_conv - True
33 conv2_block3_2_bn - True
34 conv2_block3_2_relu - True
35 conv2_block3_3_conv - True
36 conv2_block3_3_bn - True
37

In [ ]:
#finetune resnet50

model_finetuned = K.models.Sequential()
model_finetuned.add(K.layers.Lambda(lambda x: tf.image.resize(x,(224, 224))))
model_finetuned.add(resnet50)
model_finetuned.add(Flatten())
model_finetuned.add(K.layers.Dense(10, activation='softmax'))

In [ ]:
model_finetuned.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(lr=2e-5),metrics=['accuracy'])
history1=model_finetuned.fit(x_train,y_train_cat,epochs=3,validation_data=(x_test,y_test_cat))
evaluation = model_finetuned.evaluate(x_test, y_test_cat)
print('Test Accuracy: {}'.format(evaluation[1]))

model_finetuned.save("resnet50-finetuned-cifar10.h5")
model_finetuned.summary()

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/3
1563/1563 [==============================] - 576s 364ms/step - loss: 0.6544 - accuracy: 0.8052 - val_loss: 0.3672 - val_accuracy: 0.8931
Epoch 2/3
1563/1563 [==============================] - 575s 368ms/step - loss: 0.1129 - accuracy: 0.9624 - val_loss: 0.3290 - val_accuracy: 0.9038
Epoch 3/3
313/313 [==============================] - 33s 107ms/step - loss: 0.3425 - accuracy: 0.9105
Test Accuracy: 0.9104999899864197
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_2 (Lambda)           (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_5 (Dense)    

The method of ‘freezing layers’ allows a faster computation but hits the accuracy 

In [ ]:
###resnet18 and student

import tensorflow.keras.models 



resnet_input =  Input((32,32,3))
conv_1 = Conv2D(filters=16,kernel_size=(3,3),activation='relu',padding="same")(resnet_input)


conv_b1_1 = Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding="same")(conv_1)
conv_b1_2 = Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding="same")(conv_b1_1)
sum_0_2 = Concatenate()([conv_1,conv_b1_2])
conv_b1_3 = Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding="same")(sum_0_2)
conv_b1_4 = Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding="same")(conv_b1_3)

sum_1 = Concatenate()([sum_0_2,conv_b1_4])
avg_1 = AveragePooling2D(pool_size=(2,2))(sum_1)

conv_b2_1 = Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding="same")(avg_1)
conv_b2_2 = Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding="same")(conv_b2_1)
sum_1_2 = Concatenate()([avg_1,conv_b2_2])
conv_b2_3 = Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding="same")(sum_1_2)
conv_b2_4 = Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding="same")(conv_b2_3)

sum_2 = Concatenate()([sum_1_2,conv_b2_4])
avg_2 = AveragePooling2D(pool_size=(2,2))(sum_2)

conv_b3_1 = Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding="same")(avg_2)
conv_b3_2 = Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding="same")(conv_b3_1)
sum_2_2 = Concatenate()([avg_2,conv_b3_2])
conv_b3_3 = Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding="same")(sum_2_2)
conv_b3_4 = Conv2D(filters=128,kernel_size=(3,3),activation='relu',padding="same")(conv_b3_3)

sum_3 = Concatenate()([sum_2_2,conv_b3_4])
avg_3 = AveragePooling2D(pool_size=(2,2))(sum_3)

conv_b4_1 = Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding="same")(avg_3)
conv_b4_2 = Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding="same")(conv_b4_1)
sum_3_2 = Concatenate()([avg_3,conv_b4_2])
conv_b4_3 = Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding="same")(avg_3)
conv_b4_4 = Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding="same")(conv_b4_3)

sum_4 = Concatenate()([sum_3_2,conv_b4_4])
avg = AveragePooling2D(pool_size=(2,2))(sum_4)

flat = Flatten()(avg)#problema <--

dense1 = Dense(16,activation='relu')(flat)#avg
dense2 = Dense(10,activation='softmax')(flat)#maxp


flat = Flatten()(dense2)



resnet_fix = tensorflow.keras.models.Model(inputs=resnet_input,outputs=dense2)

In [ ]:
resnet_fix.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(lr=2e-5),metrics=['accuracy'])
history1=resnet_fix.fit(x_train,y_train_cat,epochs=3,validation_data=(x_test,y_test_cat))
evaluation = resnet_fix.evaluate(x_test, y_test_cat)
print('Test Accuracy: {}'.format(evaluation[1]))

resnet_fix.save("resnet18-cifar10.h5")
resnet_fix.summary()

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/3
1563/1563 [==============================] - 29s 18ms/step - loss: 1.7215 - accuracy: 0.4112 - val_loss: 1.4214 - val_accuracy: 0.4934
Epoch 2/3
1563/1563 [==============================] - 27s 17ms/step - loss: 1.3260 - accuracy: 0.5306 - val_loss: 1.2553 - val_accuracy: 0.5501
Epoch 3/3
313/313 [==============================] - 2s 6ms/step - loss: 1.1337 - accuracy: 0.6027
Test Accuracy: 0.6026999950408936
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_2[0][0]']                
                                                                   

In [ ]:
###https://keras.io/examples/vision/knowledge_distillation/


from tensorflow import keras

class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):

        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [ ]:
distiller = Distiller(student=resnet_fix, teacher=model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adam(lr=2e-5),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=3)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/3


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1563/1563 [==============================] - 184s 116ms/step - sparse_categorical_accuracy: 0.6249 - student_loss: 1.0800 - distillation_loss: 0.0224
Epoch 2/3
1563/1563 [==============================] - 189s 121ms/step - sparse_categorical_accuracy: 0.6551 - student_loss: 0.9972 - distillation_loss: 0.0209
Epoch 3/3
313/313 [==============================] - 2s 6ms/step - sparse_categorical_accuracy: 0.6761 - student_loss: 0.9322


[0.6761000156402588, 0.7653984427452087]

Knowledge Distillation is a procedure for model compression, in which a small (student) model is trained to match a large pre-trained (teacher) model. Knowledge is transferred from the teacher model to the student by minimizing a loss function, aimed at matching softened teacher logits as well as ground-truth labels.

The logits are softened by applying a "temperature" scaling function in the softmax, effectively smoothing out the probability distribution and revealing inter-class relationships learned by the teacher.